In [130]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary


In [135]:
ean_list= ['8487497376498', '8435484051026']

In [136]:
def ebay(ean_list):
    
    ebay_info = pd.DataFrame()
   
    for ean in ean_list:

        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        browser = webdriver.Firefox()
        browser.get('https://www.ebay.es/')

        search = browser.find_element_by_xpath('//input[@class="gh-tb ui-autocomplete-input"]')
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)

        action = ActionChains(browser)
        click_image  = browser.find_element_by_xpath("(//img[@class='img'])[1]")
        action.move_to_element((click_image)).click().perform()

        time.sleep(3)

        title = browser.find_elements_by_xpath('//h1[@id="itemTitle"]')[0].text
        price = browser.find_elements_by_xpath('//span[@id="prcIsum"]')[0].text
        main_image = browser.find_element_by_xpath('//img[@id="icImg"]').get_attribute('src')


        ebay_selenium = pd.DataFrame({'ean.value': [ean],
                                     'item_name.value': [title], 
                                     'list_price.value_with_tax': [price],
                                     'main_image': [main_image]
                                    })

        url = str(browser.current_url)
        time.sleep(3)
        lista = pd.read_html(url, match='.+', flavor = None, header = None, index_col =None, skiprows=None, attrs=None, parse_dates=False, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
        time.sleep()

        Detalles_tecnicos = lista[12]

        Detalles_tecnicos_1 = Detalles_tecnicos[[0, 1]]
        Detalles_tecnicos_1.set_index(0,inplace=True)
        Detalles_tecnicos_1_t = Detalles_tecnicos_1.T.reset_index(drop=True)

        Detalles_tecnicos_2 = Detalles_tecnicos[[2, 3]]
        Detalles_tecnicos_2.set_index(2,inplace=True)
        Detalles_tecnicos_2_t = Detalles_tecnicos_2.T.reset_index(drop=True)

        Detalles_tecnicos_t= pd.concat([Detalles_tecnicos_1_t, Detalles_tecnicos_2_t], axis=1, sort=False)

        browser.close()
        
        ebay_data = pd.concat([ebay_selenium, Detalles_tecnicos_t], axis=1, sort=False)
        ebay_data = ebay_data.set_index('ean.value')

        ebay_info = pd.concat([ebay_info, ebay_data], axis=1, sort=False)

            
    return  ebay_info




In [137]:
df = ebay(ean_list)

In [138]:
df

,item_name.value,list_price.value_with_tax,main_image,Estado:,MPN:,CaracterÃ­sticas:,Fuente de alimentaciÃ³n:,Voltaje:,GarantÃ­a del fabricante:,Tiempo de calentamiento:,...,Potencia:,Color:,EAN:,item_name.value,list_price.value_with_tax,main_image,Estado:,Tipo:,Marca:,EAN:
8487497376498,Plancha de Vapor de Máxima Potencia 3.200w . X...,"23,99 EUR c/u",https://i.ebayimg.com/images/g/ZZkAAOSwHwReKu4...,"Nuevo: Un artÃ­culo nuevo, sin usar, sin abrir...",8487497376498,"Antiadherente, AnticalcÃ¡reo, Antigoteo, Auto-...",220/230v,220 V,2 aÃ±os,10 segundos,...,3200 W,Verde,8487497376795,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8435484051026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Cecotec 3D ForceAnodized 550 - Plancha de Vapo...,"31,92 EUR",https://i.ebayimg.com/images/g/zOIAAOSwlf9df6F...,"Nuevo: Un artÃ­culo nuevo, sin usar, sin abrir...",Plancha a vapor,Cecotec,8435484051026
